In [16]:
import Bio
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [17]:
#region_with_L1HS = SeqIO.read('ref_region_with_L1HS.fa', 'fasta')

In [18]:
#L1_start_pos = 73002 #note I subtracted one from both to account for Boissinot's likely use of base 1 indexing
#L1_end_pos = 79019

In [19]:
#L1asfeature = SeqFeature(FeatureLocation(L1_start_pos, L1_end_pos), type='gene', strand = -1)

In [20]:
#L1HS = L1asfeature.extract(region_with_L1HS)

In [21]:
#L1HS

SeqRecord(seq=Seq('TTATACTCTAAGTTTTAGGGTACATGTGCACATTGTGCAGGTTAGTTACATATG...CCC'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[])

In [22]:
#L1HS.id = 'L1PA1_in_AC004554'

In [23]:
#diagnostic_nucleotide_feature = SeqFeature(location=FeatureLocation(start = 5955, end = 5956), type='snp', id = 'diagnostic_nt' )

In [24]:
#L1HS.features.append(diagnostic_nucleotide_feature)

In [25]:
#L1HS.features

In [26]:
#L1HS.features[0].extract(L1HS.seq)

In [27]:
#L1HS.seq[5950:5960]

In [44]:
L1HS3prime = Bio.SeqIO.read('L1HS_3prime_DFAM.fa', 'fasta')

In [51]:
L1HS3prime = L1HS3prime.upper()

In [129]:
non_ta_seq = Seq('ACGTTAGTGGGTGC') #see boissinot 2000 fig 4
ta_seq = Seq('ACATTAGTGGGTGC')
downstream_diagnostic_nuc_seq = Seq('TTAGTGGGTGC')

In [130]:
non_ta_2 = Seq("GATGACGAG")

In [131]:
test_seq = Seq("CCTAATGCTAGATGACACA")

In [183]:

def GetDiagnosticNuc(read_seq, target_seq = 'TTAGTGGGTGC'):
    """
    input:
    target must be a Seq (by default it is 11 nuc 3' of diagnostic nuc in boissinot), read must be a Seq or string
    
    output:
    returns one of the following lists:
    ['likely_ta', read_seq[ACpos:diagnostic_nuc_read_pos+1]
    ['likely_non_ta', read_seq[ACpos:diagnostic_nuc_read_pos+1]
    ['no_diagnostic_info_in_read']
    """
    #import and define aligner parameters
    from Bio import Align

    aligner = Align.PairwiseAligner()
    aligner.mode = 'local'
    aligner.match_score = 2
    aligner.mismatch_score = -1
    aligner.open_gap_score = -0.5
    aligner.extend_gap_score = -0.1
    aligner.target_end_gap_score = 0.0
    aligner.query_end_gap_score = 0.0

    #retrieve diagnostic nuc in reads which have a perfect
    #alignment to the sequence immediately downstream of diag nuc

    ta_supporting_read = []
    non_ta_supporting_read = []
    no_info_read = []


    for alignment in aligner.align(target_seq, read_seq):
        print("Score = %.1f:" % alignment.score)
        print(alignment)
        print(alignment.aligned)
        if alignment.score ==22:
        #this means the read has a perfect match to region right after diag nuc
        #perfect match requirement may be too stringent
            diagnostic_nuc_read_pos = alignment.aligned[1][0][0] - 1
            ACpos = alignment.aligned[1][0][0] - 3
            if ACpos >=0:
                ACseq = read_seq[ACpos:diagnostic_nuc_read_pos]
                diagnostic_nuc_read = read_seq[diagnostic_nuc_read_pos]
                print(ACseq+diagnostic_nuc_read)
                if ACseq == 'AC':
                    if diagnostic_nuc_read == 'A':
                        return ['likely_ta', read_seq[ACpos:diagnostic_nuc_read_pos+1]]
                    elif diagnostic_nuc_read == 'G':
                        return ['likely_non_ta', read_seq[ACpos:diagnostic_nuc_read_pos+1]]
                    else:
                        return ['diagnostic nuc not A or G']
                else:
                    return ['AC nucleotides absent']
                
            else: 
                return ["""no_diagnostic_info_in_read; read does not extend far enough to see ACA/G"""]
        else:
            return ['No perfect alignment with target seq in read']




In [185]:
GetDiagnosticNuc('ACATTAGTGGGTGC')

Score = 19.0:
   TTAGTGGGTGC
   |||||||||.|
ACATTAGTGGGTAC

(((0, 11),), ((3, 14),))


['No perfect alignment with target seq in read']